# STAT 344 GROUP PROJECT



In [1]:
# Run this cell before continuing.
library(dplyr)
library(ggplot2)
library(tibble)
library(infer)




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
student_df<-read.csv("https://raw.githubusercontent.com/Yuji03b/STAT-344-group-project/main/student_data.csv",header=T)

###  1. Our first parameter is : avergae final grade G3 

In [3]:
n <- 100             #(sample size
N <- nrow(student_df)  # population size

female_data <- subset(student_df,sex=="F")  # the female population 
male_data <- subset(student_df,sex=="M")  # the male population

female_N<-nrow(female_data)    # the female student population size
male_N<-nrow(male_data)      # the male student population size

###  1.1 Determine sample size

### 1.1 Stratified sample :using proportional allocation  (strata= gender)

In [4]:
#using proportional allocation N_k/N and take sample from each sub-population
set.seed(12345)
female_n <- round(n*(female_N/N))
male_n <- round(n*(male_N/N))
female_sample <- sample_n(female_data,size=female_n,replace=F)
male_sample <- sample_n(male_data,size=male_n,replace=F)

In [5]:
 # estimate of stratified mean
stratified_mean_grade <- mean(female_sample$G3)*(female_N/N)+mean(male_sample$G3)*(male_N/N) 

# standard error of stratified mean grade
female_FPC <- 1-female_n/female_N
male_FPC <- 1-male_n/male_N

se_stratified<-sqrt((female_N/N)^2*female_FPC*var(female_sample$G3)/female_n+
                   (male_N/N)^2*male_FPC*var(male_sample$G3)/male_n)
#95% confidence interval
stratified_CI<- c(stratified_mean_grade - 1.96* se_stratified,stratified_mean_grade + 1.96* se_stratified)

In [6]:
#check within variance roungly the same we will use proportional allocation.
within_variance_female<-(female_N/N)*var(female_data$G3)
within_variance_male<-(male_N/N)*var(male_data$G3)
cbind(within_variance_female,within_variance_male)

within_variance_female,within_variance_male
11.25096,9.56668


### 1.2 Stratified sample :using optimal allocation (strata = gender)

In [16]:
c = 100/(male_N * sd(male_data$G3) + female_N * sd(female_data$G3) )
n_female_optimal = round(c* female_N * sd(female_data$G3),0)
n_male_optimal = round(c* male_N * sd(male_data$G3),0)

# n_female_optimal is optimal allocation female size
# female_n is proportional allocation female size

cbind(n_female_optimal,female_n)

n_female_optimal,female_n
53,53


### 1.3 using school strata or gender strata ?

In [9]:
n <- 100             #(sample size
N <- nrow(student_df)  # population size

GP_data <- subset(student_df,school=="GP")  # the GP school population 
MS_data <- subset(student_df,school=="MS")  # the MS school population

#between-stratified variance of school strata

between_variance_of_school<-(nrow(GP_data)/N)*(mean(GP_data$G3)-mean(student_df$G3))^2+
                            (nrow(MS_data)/N)*(mean(MS_data$G3)-mean(student_df$G3))^2

between_variance_of_gender <- (female_N/N)*(mean(female_data$G3)-mean(student_df$G3))^2+
                                (male_N/N)*(mean(male_data$G3)-mean(student_df$G3))^2


cbind(between_variance_of_school,
between_variance_of_gender)

between_variance_of_school,between_variance_of_gender
0.0424283,0.2240846


### Random select sample (SRS) for average G3

In [10]:
set.seed(12345)
SRS_sample <- sample_n(student_df,size=n,replace=F)
SRS_FPC <- (1-n/N)

SRS_mean_grade <-mean(SRS_sample$G3) # estimate mean of SRS
se_SRS <- sqrt(SRS_FPC*var(SRS_sample$G3)/n) #standard error of SRS estimate

SRS_CI<- c(SRS_mean_grade - 1.96* se_SRS,SRS_mean_grade + 1.96* se_SRS)

In [11]:
average_G3_score <- c(SRS_mean_grade,stratified_mean_grade)
sampling_method <- c("SRS","stratified method")
standard_error <- c(se_SRS,se_stratified)


good_table <- data.frame("sampling method"=sampling_method,
                         "average G3 score"=average_G3_score,
                         "standard error"=standard_error,
                         "CI lower" = c(SRS_CI[1],stratified_CI[1]),
                         "CI upper" = c(SRS_CI[2],stratified_CI[2]))
good_table

sampling.method,average.G3.score,standard.error,CI.lower,CI.upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SRS,9.97000,0.3975370,9.190827,10.74917
stratified method,11.09977,0.3680666,10.378356,11.82118


### 2. The second parmeter is : proportion of student who get 80% score in G3 (16 out of 20)


In [17]:
A_score_df <- subset(student_df,(G3<=20)&(G3>=16))
(nrow(A_score_df)/N) *n   #check if np >10
(1-nrow(A_score_df)/N) *n   #check if n(1-p) >10

[1] 10.12658

[1] 89.87342

### 2.1 SRS for A score (80%)

In [13]:
SRS_proportion_Ascore <- nrow(subset(SRS_sample,(G3<=20)&(G3>=16)))/n 
se_SRS_Ascore <- sqrt(SRS_FPC*SRS_proportion_Ascore*(1-SRS_proportion_Ascore)/n) #standard error of SRS estimate

SRS_CI_Ascore<- c(SRS_proportion_Ascore - 1.96* se_SRS_Ascore,SRS_proportion_Ascore + 1.96* se_SRS_Ascore)


cbind(SRS_proportion_Ascore,
se_SRS_Ascore)

SRS_proportion_Ascore,se_SRS_Ascore
0.08,0.02344506


### 2.2 Stratified sampling (proportional allocation) for A score

In [18]:
female_Ascore <- nrow(subset(female_sample,(G3<=20)&(G3>=16)))/female_n
male_Ascore <- nrow(subset(male_sample,(G3<=20)&(G3>=16)))/male_n

stratified_proportion_Ascore <- female_Ascore*(female_N/N)+male_Ascore*(male_N/N)

se_stratified_Ascore <-sqrt((female_N/N)^2*female_FPC*(female_Ascore*(1-female_Ascore))/female_n+
                   (male_N/N)^2*male_FPC*(male_Ascore*(1-male_Ascore))/male_n)
stratified_CI_Ascore<- c(stratified_proportion_Ascore - 1.96* se_stratified_Ascore,
                         stratified_proportion_Ascore + 1.96* se_stratified_Ascore)


#### 2.2.1 check between variance of gender and school

In [21]:
#gender variance 

female_A_true_p <- nrow(subset(female_data,(G3<=20)&(G3>=16)))/nrow(female_data)
male_A_true_p <- nrow(subset(male_data,(G3<=20)&(G3>=16)))/nrow(male_data)
true_p <- nrow(subset(student_df,(G3<=20)&(G3>=16)))/N


between_variance_gender <- (nrow(female_data)/N)*(female_A_true_p-true_p)^2+
                            (nrow(male_data)/N)*(male_A_true_p-true_p)^2

GP_A_true_p <- nrow(subset(GP_data,(G3<=20)&(G3>=16)))/nrow(GP_data)
MS_A_true_p <- nrow(subset(MS_data,(G3<=20)&(G3>=16)))/nrow(MS_data)



between_variance_of_school<-(nrow(GP_data)/N)*(GP_A_true_p-true_p)^2+
                            (nrow(MS_data)/N)*(MS_A_true_p-true_p)^2


cbind(between_variance_gender,between_variance_of_school)

between_variance_gender,between_variance_of_school
0.0006591145,0.0001712794


### 2.2.2 check optimal allocation

In [23]:

c = 100/(male_N * sqrt(male_A_true_p*(1-male_A_true_p) )+ 
         female_N * sqrt(female_A_true_p*( 1-female_A_true_p)))

n_female_optimal_A = round(c* female_N * sqrt(female_A_true_p*( 1-female_A_true_p)),0)
n_male_optimal_A = round(c* male_N * sqrt(male_A_true_p*( 1-male_A_true_p)),0)

# n_female_optimal_A is  optical allocation female size
# female_n is proportional allocation female size

cbind(n_female_optimal_A,female_n)

n_female_optimal_A,female_n
47,53


### 2.3 Stratified sampling (optimal allocation) for A score

In [26]:
set.seed(12345)
female_sample2 <- sample_n(female_data,size=n_female_optimal_A,replace=F)
male_sample2 <- sample_n(male_data,size=n_male_optimal_A,replace=F)

female_FPC2 <- (1-n_female_optimal_A/female_N)
male_FPC2 <- (1-n_male_optimal_A/male_N)

female_Ascore_op <- nrow(subset(female_sample2,(G3<=20)&(G3>=16)))/nrow(female_sample2)
male_Ascore_op <- nrow(subset(male_sample2,(G3<=20)&(G3>=16)))/nrow(male_sample2)

stratified_proportion_Ascore_op <- female_Ascore_op*(female_N/N)+male_Ascore_op*(male_N/N)

se_stratified_Ascore_op <-sqrt((female_N/N)^2*female_FPC2*(female_Ascore_op*(1-female_Ascore_op))/nrow(female_sample2)+
                   (male_N/N)^2*male_FPC2*(male_Ascore_op*(1-male_Ascore_op))/nrow(male_sample2))
stratified_CI_Ascore_op<- c(stratified_proportion_Ascore_op - 1.96* se_stratified_Ascore_op,
                            stratified_proportion_Ascore_op + 1.96* se_stratified_Ascore_op)

### Conclusion table 2

In [27]:
sampling_method2 <- c("SRS","Stratification with proportional alloction ","Stratification with optimal alloction")
A_prop <- c(SRS_proportion_Ascore,stratified_proportion_Ascore,stratified_proportion_Ascore_op)
standard_error_A <- c(se_SRS_Ascore,se_stratified_Ascore,se_stratified_Ascore_op)
good_table_2 <- data.frame("sampling method"=sampling_method2,"proportion of A score"=A_prop,"standard error"=standard_error_A,
                         "CI lower" = c(SRS_CI_Ascore[1],stratified_CI_Ascore[1],stratified_CI_Ascore_op[1]),
                           "CI upper" = c(SRS_CI_Ascore[2],stratified_CI_Ascore[2],stratified_CI_Ascore_op[2]))
good_table_2

sampling.method,proportion.of.A.score,standard.error,CI.lower,CI.upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SRS,0.08000000,0.02344506,0.03404768,0.1259523
Stratification with proportional alloction,0.09990396,0.02587237,0.04919412,0.1506138
Stratification with optimal alloction,0.11188532,0.02770299,0.05758746,0.1661832
